In [24]:
# Parameter setting 
import math
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB
import random

# This model is referred and modified from :
# Beraldi, Patrizia, et al. "Rolling-horizon and fix-and-relax heuristics for the parallel machine lot-sizing and scheduling problem with sequence-dependent set-up costs." Computers & Operations Research 35.11 (2008): 3644-3656.

# define the parameter 
random.seed(1)
c_ij = {}
D_it= {}
number_of_machine = 480  
number_of_item = 22     
time_period =  1000 

_sum_demand = 0
for i in range(0, number_of_item):
    for j in range(0, number_of_item):
        c_ij[i,j]=  random.randrange(12, 18)
        
for i in range(0, number_of_item):
    for t in range(0, time_period):
        D_it[i,t] = random.randrange(3, 32) 
        _sum_demand += D_it[i,t]
        
# setting the length of the subperiod :
sub_len = 200
print('Finsh setting up the parameter')
print('Demand = ',_sum_demand)
print('Capacity = ', number_of_machine*time_period)
print('Utilization(%) = {:.0%}'.format(_sum_demand/(number_of_machine*time_period)))

Finsh setting up the parameter
Demand =  373337
Capacity =  480000
Utilization(%) = 78%


In [2]:
# setting the function which using to create the model 
def partition_period(period,sub_period_length):
    partition_period,_cnt,_cnt_period = {},0,0
    for i in range(period):
        if _cnt <= sub_period_length -1:
            try:
                partition_period[_cnt_period].append(i)
            except KeyError:
                partition_period[_cnt_period] = [i]
            _cnt+=1
        else:
            _cnt_period+=1 
            partition_period[_cnt_period] = [i]
            _cnt=1
    return partition_period

def transfer_period_into_interval(partition_period):
    trans = {}
    for r in partition_period:
        for r1 in partition_period[r]:
            trans[r1] = r
    return trans

def generate_due_date(D_head,number_of_item,partition_period,time_period):
    L_ir = {}
    for i in range(0,number_of_item):
        for r in range(0,len(partition_period)-1):
            for l in partition_period[r+1]: 
                if D_head[r+1,i,l] >0 :
                    try:
                        L_ir[i,r].append(l)
                    except KeyError:                        
                        L_ir[i,r]= [l]
    for r in range(0,len(partition_period)-1):
        for r1 in range(r+1,len(partition_period)-1):
            for i in range(0,number_of_item):
                try:
                    L_ir[i,r] = L_ir[i,r]+L_ir[i,r1] 
                except KeyError:
                    pass
    return L_ir

In [3]:
# using the function which created by previous segment:
partition_period = partition_period(time_period,sub_len)    
trans = transfer_period_into_interval(partition_period )

In [16]:
# create the original model :
original_model_obj = 0
m = gp.Model()
# create the decision variable:
y_ijt = m.addVars(number_of_item,number_of_item,time_period,lb= 0,vtype = GRB.CONTINUOUS, name = "y")
x_it = m.addVars(number_of_item,time_period,lb= 0,vtype = GRB.INTEGER, name = "x")
m_i = m.addVars(number_of_item,lb= 0, ub = number_of_machine + 0.02,vtype = GRB.INTEGER, name = "m")
# set the obj. func. :       
m.setObjective(gp.quicksum(c_ij[i,j] * y_ijt[i,j,t] for i in range(0,number_of_item) for j in range(0,number_of_item) for t in range(0,time_period) ),GRB.MINIMIZE)
# create constraints:
m.addConstr(gp.quicksum(m_i[i] for i in range(0,number_of_item ) ) == number_of_machine,"number_of machine"  )

for i in range(0, number_of_item):
    m.addConstr(x_it[i,0] - gp.quicksum(y_ijt[j,i,0] for j in range(0,number_of_item) if j != i ) + gp.quicksum(y_ijt[i,j,0] for j in range(0,number_of_item) if j != i ) == m_i[i],"ini %s" %(i))

for i in range(0, number_of_item):
        for t in range (1,time_period):
            m.addConstr(x_it[i,t] - gp.quicksum(y_ijt[j,i,t] for j in range(0,number_of_item) if j != i ) + gp.quicksum(y_ijt[i,j,t] for j in range(0,number_of_item) if j != i ) == x_it[i,t-1],"flow %s , %s" %(i,t))

for i in range(0, number_of_item):
    for t in range (0,time_period):
        m.addConstr(gp.quicksum(x_it[i,t1] for t1 in range(0,t+1)) >= gp.quicksum(D_it[i,t1] for t1 in range(0,t+1)),"demand %s , %s" %(i,t))

m.update()
m.Params.timeLimit = 1500.0
m.optimize()
original_model_obj=m.objval

Set parameter TimeLimit to value 1500
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 44001 rows, 506022 columns and 11979022 nonzeros
Model fingerprint: 0x32e92071
Variable types: 484000 continuous, 22022 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [5e+02, 5e+02]
  RHS range        [3e+00, 2e+04]
Presolve removed 22 rows and 22000 columns (presolve time = 5s) ...
Presolve removed 22 rows and 22000 columns
Presolve time: 9.17s
Presolved: 43979 rows, 484022 columns, 11979000 nonzeros
Variable types: 462000 continuous, 22022 integer (0 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Elapsed ordering time = 5s
Elapsed ordering time = 10s
Elapsed ordering time = 15s
Elapsed ordering time = 17s
Elapsed ordering t

In [6]:
# Rolling optimization algorithm:
# Create the initial problem :
initialProblem = gp.Model('initialProblem')
x_it = initialProblem.addVars(number_of_item ,time_period,vtype =GRB.INTEGER,name = "x")
y_ijt = initialProblem.addVars(number_of_item,number_of_item,time_period,vtype = GRB.CONTINUOUS, name = "y")
m_i =initialProblem.addVars(number_of_item,lb= 0, ub = number_of_machine + 0.02,vtype = GRB.INTEGER, name = "m")

# set the objective function:
initialProblem.setObjective(gp.quicksum(c_ij[i,j] * y_ijt[i,j,t] for i in range(0,number_of_item) for j in range(0,number_of_item) for t in partition_period[0]),GRB.MINIMIZE)

# set up constraint 1:
initialProblem.addConstr(gp.quicksum(m_i[i] for i in range(0,number_of_item ) ) == number_of_machine,"number_of machine" )

# set up constraint 2:
for i in range(0, number_of_item):
    initialProblem.addConstr(x_it[i,0] - gp.quicksum(y_ijt[j,i,0] for j in range(0,number_of_item) if j != i ) + gp.quicksum(y_ijt[i,j,0] for j in range(0,number_of_item) if j != i ) == m_i[i],"ini %s" %(i))

# set up constraint 3:    
for i in range(0, number_of_item):
        for t in range (1,partition_period[0][-1]+1):
            initialProblem.addConstr(x_it[i,t] - gp.quicksum(y_ijt[j,i,t] for j in range(0,number_of_item) if j != i ) + gp.quicksum(y_ijt[i,j,t] for j in range(0,number_of_item) if j != i ) == x_it[i,t-1],"flow %s , %s" %(i,t))
# constraints 9:
for r in range(1,len(partition_period)):
    for i in range(0,number_of_item):
        initialProblem.addConstr(x_it[i,partition_period[r][0]]-gp.quicksum(y_ijt[j,i,partition_period[r][0]] for j in range(0,number_of_item) if j != i) + gp.quicksum(y_ijt[i,j,partition_period[r][0]] for j in range(0,number_of_item) if j != i)  ==x_it[i,partition_period[r-1][0]],"period flow constraint 9 %s,%s" % (r,i)  )
# set up extra constraint:
for t in range(0,time_period):
    initialProblem.addConstr(gp.quicksum(x_it[i,t] for i in range(0,number_of_item)) <= number_of_machine,"machine upper bound %s" %(t) )

# setting up dit head
D_rit_head ={}

for r in partition_period:
    for t in partition_period[r]:
        for i in range(0,number_of_item):
            D_rit_head[r,i,t] = D_it[i,t]

# setting up the constraint 10:
for r in partition_period:
    for l in partition_period[r]:
        for i in range(0,number_of_item):
            initialProblem.addConstr(gp.quicksum(x_it[i,t] for t in range(partition_period[r][0],min(l+1,partition_period[r][-1]+1) ) ) >= gp.quicksum(D_rit_head[r,i,t] for t in range(partition_period[r][0],min(l+1,partition_period[r][-1]+1) ) ),"constraint 10 (%s,%s,%s)" %(r,l,i) )

# create the list Lir:
L_ir = generate_due_date(D_rit_head,number_of_item,partition_period,time_period)
u_il = initialProblem.addVars(number_of_item,time_period, lb = 0, vtype = GRB.INTEGER, name = "u")
# construct constraints 12:
for r in range(0, len(partition_period)-1):
    for i in range(0,number_of_item):
        for l in L_ir[i,r]:           
            initialProblem.addConstr(gp.quicksum(x_it[i,t] for t in range(partition_period[r][0],partition_period[r][-1]+1)) + gp.quicksum(u_il[i,t] for t in L_ir[i,r] if t <= l) >= gp.quicksum(D_rit_head[trans[t],i,t] for t in range(partition_period[r+1][0],l+1)),"constraint 12 (%s,%s,%s)" %(r,l,i))
# construct constraints 13:
for r in range(0, len(partition_period)-1):
    for i in range(0,number_of_item):
        for l in L_ir[i,r]:            
            initialProblem.addConstr(gp.quicksum(u_il[i1,t] for i1 in range(0,number_of_item) for t in L_ir[i,r] if t <= l )<= number_of_machine*(l-partition_period[r+1][0]+1),"constraint 13 (%s,%s,%s)" %(r,l,i)  )  
    
initialProblem.update()
initialProblem.Params.timeLimit = 1200.0
initialProblem.optimize()


Set parameter TimeLimit to value 1200
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 115489 rows, 528022 columns and 315336494 nonzeros
Model fingerprint: 0x992c83b8
Variable types: 484000 continuous, 44022 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [5e+02, 5e+02]
  RHS range        [3e+00, 4e+05]
Presolve removed 0 rows and 0 columns (presolve time = 49s) ...
Presolve removed 0 rows and 0 columns (presolve time = 50s) ...
Presolve removed 0 rows and 394152 columns (presolve time = 83s) ...
Presolve removed 0 rows and 394152 columns (presolve time = 206s) ...
Presolve removed 110 rows and 394152 columns (presolve time = 242s) ...
Presolve removed 110 rows and 394152 columns (presolve time = 272s) ...
Presolve removed 173 rows and 394152 columns (presolve time = 276s) ...
Presolve removed 173 rows

In [13]:
# intial solution store in ini_x for check the feasibility of initial solution
_x =0
_d = 0
ini_x={}
initial_feasible = True
# # inital solution 存進  ini_x 裡面
for element in partition_period[0]:
    for i in range(0,number_of_item):
        ini_x[i,element] = x_it[i,t].X
        _x +=ini_x[i,element]
        _d +=D_it[i,element]
        if _x < _d:
            initial_feasible = False
            print('infeasible')
if initial_feasible == True:
    print('The initial solution of rolling horizion is feasibile! ')

The initial solution of rolling horizion is feasibile


In [14]:
# cerate problem 1 to problem k
obj = []
obj.append(initialProblem.objval)
_xremain = {}
for r_loop in range (1, len(partition_period)):
    x_it_value = {}
    for i in range(0,number_of_item):
        for t in range(0,time_period):
            if t>= partition_period[r_loop][0]:
                x_it_value[i,t]=0
            else:
                x_it_value[i,t] = x_it[i,t].x
                                
    direction = True 
    # if the produce quantity > demand in the period, 
    # then the surplus quantity can be assign into next period
    remain_capa = 0
    _sum_capa =  len(partition_period[r_loop])*number_of_machine
    for i in range(0,number_of_item):
        for t in range(partition_period[r_loop][0],partition_period[r_loop][-1]+1):
            _sum_capa = _sum_capa -x_it_value[i,t]
        remain_capa = max(0,_sum_capa)
    if r_loop + 1 <= len(partition_period)-1:
        for i in range(0,number_of_item):
            for r in range(r_loop+1, len(partition_period)):
                if direction == False:
                    break
                else:
                    for t in partition_period[r]:
                        if remain_capa > 0: # has reamin capacity can be assign
                            if remain_capa > D_it[i,t]:
                                D_rit_head[r,i,t] =0
                                remain_capa = remain_capa - D_it[i,t]
                            else:
                                D_rit_head[r,i,t] = D_it[i,t] - remain_capa
                                remain_capa= 0 
                                direction = False
                                break
    else:
        pass

    # update the list Lir:
    L_ir = generate_due_date(D_rit_head,number_of_item,partition_period,time_period)
    # create problem r_loop model:
    subProblem = gp.Model('subProblem')
    x_it = subProblem.addVars(number_of_item ,time_period,vtype =GRB.INTEGER,name = "x")
    y_ijt = subProblem.addVars(number_of_item,number_of_item,time_period,vtype = GRB.CONTINUOUS, name = "y")
    m_i =subProblem.addVars(number_of_item,lb= 0, ub = number_of_machine + 0.02,vtype = GRB.INTEGER, name = "m")
    
    # constraints 9 new :
    for r in range(r_loop,len(partition_period)):
        for i in range(0,number_of_item):
            if r ==r_loop:
                subProblem.addConstr(x_it[i,partition_period[r][0]]-gp.quicksum(y_ijt[j,i,partition_period[r][0]] for j in range(0,number_of_item) if j != i) + gp.quicksum(y_ijt[i,j,partition_period[r][0]] for j in range(0,number_of_item) if j != i)  ==x_it_value[i,partition_period[r-1][-1]],"period flow constraint 9 %s,%s" % (r,i)  )
            else:
                subProblem.addConstr(x_it[i,partition_period[r][0]]-gp.quicksum(y_ijt[j,i,partition_period[r][0]] for j in range(0,number_of_item) if j != i) + gp.quicksum(y_ijt[i,j,partition_period[r][0]] for j in range(0,number_of_item) if j != i)  ==x_it[i,partition_period[r-1][-1]],"period flow constraint 9 %s,%s" % (r,i)  )
    # constraint 3 for subperiod tr-> Tr:
    # set up constraint 3:    
    for i in range(0, number_of_item):
            for t in range (partition_period[r_loop][1],partition_period[r_loop][-1]+1):
                subProblem.addConstr(x_it[i,t] - gp.quicksum(y_ijt[j,i,t] for j in range(0,number_of_item) if j != i ) + gp.quicksum(y_ijt[i,j,t] for j in range(0,number_of_item) if j != i ) == x_it[i,t-1],"flow %s , %s" %(i,t))

    # constraint 10 :
    # setting up the constraint 10:
    for r in range(r_loop,len(partition_period)):
        for l in partition_period[r]:
            for i in range(0,number_of_item):
                subProblem.addConstr(gp.quicksum(x_it[i,t] for t in range(partition_period[r_loop][0],min(l+1,partition_period[r][-1]+1) ) ) >= D_rit_head[r,i,l] ,"constraint 10 (%s,%s,%s)" %(r,l,i) )
    u_il = subProblem.addVars(number_of_item,time_period, lb = 0, vtype = GRB.INTEGER, name = "u")
  
    # construct constraints 12:
    for r in range(r_loop, len(partition_period)-1):
        for i in range(0,number_of_item):
            try: 
                for l in L_ir[i,r]:  
                    subProblem.addConstr(gp.quicksum(x_it[i,t] for t in range(partition_period[r][0],partition_period[r][-1]+1)) + gp.quicksum(u_il[i,t] for t in L_ir[i,r] if t <= l) >= gp.quicksum(D_rit_head[trans[t],i,t] for t in range(partition_period[r+1][0],l+1)),"constraint 12 (%s,%s,%s)" %(r,l,i))
            except KeyError:
                pass
            
    # construct constraints 13:
    for r in range(r_loop, len(partition_period)-1):
        for i in range(0,number_of_item):
            try: 
                for l in L_ir[i,r]:
                    subProblem.addConstr(gp.quicksum(u_il[i1,t] for i1 in range(0,number_of_item) for t in L_ir[i,r] if t <= l )<= number_of_machine*(l-partition_period[r+1][0]+1),"constraint 13 (%s,%s,%s)" %(r,l,i)  )  
            except KeyError:
                pass
            
    # build the objective function 
    subProblem.setObjective(gp.quicksum(c_ij[i,j] * y_ijt[i,j,t] for i in range(0,number_of_item) for j in range(0,number_of_item) for t in partition_period[r_loop]),GRB.MINIMIZE)
    subProblem.update()
    subProblem.Params.timeLimit = 850.0
    subProblem.optimize()
    
    _xs= 0
    _ds = 0
    for t in partition_period[r_loop]:
        for i in range(0,number_of_item):
            _xremain[i,t] =  x_it[i,t].X
    obj.append(subProblem.objval)

Set parameter TimeLimit to value 850
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 51914 rows, 528022 columns and 89348932 nonzeros
Model fingerprint: 0x4e96d911
Variable types: 484000 continuous, 44022 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [5e+02, 5e+02]
  RHS range        [3e+00, 3e+05]
Presolve removed 20902 rows and 422526 columns (presolve time = 9s) ...
Presolve removed 20912 rows and 422526 columns (presolve time = 22s) ...
Presolve removed 20934 rows and 422526 columns (presolve time = 28s) ...
Presolve removed 22981 rows and 422526 columns (presolve time = 30s) ...
Presolve removed 22981 rows and 422526 columns (presolve time = 36s) ...
Presolve removed 22981 rows and 422570 columns (presolve time = 41s) ...
Presolve removed 36381 rows and 423263 columns (presolve time = 48s) ...
P

In [18]:
# Check the feasibility of the rolling horizion optimization 
_x =0
_d = 0
feasible = True
for r in partition_period:
    for element in partition_period[r]:
        if r== 0:
            for i in range(0,number_of_item):
                _x+= ini_x[i,element]
                _d +=D_it[i,element]
                if _x < _d:
                    print('infeasible')
                    feasible = False
        else:
            for i in range(0,number_of_item):
                _x+= _xremain[i,t]
                _d +=D_it[i,element]   
                if _x < _d:
                    print('infeasible')
                    feasible = False
if feasible == True:
    print('The solution of rolling horizion is feasibile and the obj. solution is: ', sum(obj))
    print('The solution of original formulation is: ', original_model_obj)

The solution of rolling horizion is feasibile and the obj. solution is:  1344.999999999996
The solution of original formulation is:  16636878.0
